In [1]:
import calliope
import pandas as pd
import plotly.express as px
import helper_funcs # load the custom helper function

calliope.set_log_verbosity("INFO", include_solver_output=False)

In [2]:
model = calliope.Model('model.yaml')

[2025-07-30 14:37:36] INFO     Model: initialising
[2025-07-30 14:37:36] INFO     Model: preprocessing stage 1 (model_run)
[2025-07-30 14:37:37] INFO     Model: preprocessing stage 2 (model_data)
[2025-07-30 14:37:37] INFO     Model: preprocessing complete


In [3]:
model.inputs

<xarray.Dataset> Size: 186kB
Dimensions:                 (costs: 1, techs: 7, nodes: 2, carriers: 3,
                             timesteps: 744)
Coordinates:
  * costs                   (costs) object 8B 'monetary'
  * techs                   (techs) object 56B 'aluminium_demand' ... 'renewa...
  * carriers                (carriers) object 24B 'aluminium' ... 'fuel'
  * nodes                   (nodes) <U11 88B 'Iceland' 'Netherlands'
  * timesteps               (timesteps) datetime64[ns] 6kB 2005-01-01 ... 200...
Data variables: (12/20)
    bigM                    float64 8B 1e+06
    objective_cost_weights  (costs) float64 8B 1.0
    base_tech               (techs) object 56B 'demand' ... 'supply'
    carrier                 (nodes, techs) object 112B nan nan nan ... nan nan
    carrier_out             (nodes, techs, carriers) float64 336B nan ... nan
    color                   (techs) object 56B '#072486' '#8E2999' ... '#F9CF22'
    ...                      ...
    sink_use_max            (timesteps, nodes, techs) float64 83kB nan ... nan
    source_use_max          (timesteps, nodes, techs) float64 83kB nan ... nan
    definition_matrix       (nodes, techs, carriers) bool 42B False ... False
    distance                (techs) float64 56B nan nan 2.021e+03 ... nan nan
    timestep_resolution     (timesteps) float64 6kB 1.0 1.0 1.0 ... 1.0 1.0 1.0
    timestep_weights        (timesteps) float64 6kB 1.0 1.0 1.0 ... 1.0 1.0 1.0
Attributes:
    calliope_version_defined:      0.7.0
    calliope_version_initialised:  0.7.0.dev6
    applied_overrides:             
    scenario:                      None
    defaults:                      {'bigM': 1000000000.0, 'objective_cost_wei...
    allow_operate_mode:            1
    name:                          main model
    timestamp_model_creation:      1753879056.755022

In [4]:
model.inputs.flow_cap_max.to_series().dropna()

techs
aluminium_transport_tech    3000.0
fuel_consumption_tech          7.0
fuel_supply                    inf
renewable_electricity          inf
Name: flow_cap_max, dtype: float64

In [5]:
model.build(force=True)
model.solve()

[2025-07-30 14:37:37] INFO     Model: backend build starting
[2025-07-30 14:37:37] INFO     Math preprocessing | added file 'plan'.
[2025-07-30 14:37:37] INFO     Math preprocessing | added file 'custom_constraints_state.yml'.
[2025-07-30 14:37:37] INFO     Math preprocessing | added file 'custom_constraints_delay.yml'.
[2025-07-30 14:37:37] INFO     Math preprocessing | validated math against schema.
[2025-07-30 14:37:37] INFO     Optimisation Model | parameters | Generated.
[2025-07-30 14:37:39] INFO     Optimisation Model | Validated math strings.
[2025-07-30 14:37:39] INFO     Optimisation Model | variables | Generated.
[2025-07-30 14:37:40] INFO     Optimisation Model | global_expressions | Generated.
[2025-07-30 14:37:41] INFO     Optimisation Model | constraints | Generated.
[2025-07-30 14:37:41] INFO     Optimisation Model | piecewise_constraints | Generated.
[2025-07-30 14:37:41] INFO     Optimisation Model | objectives | Generated.
[2025-07-30 14:37:41] INFO     Model: backen

In [6]:
model.results

<xarray.Dataset> Size: 2MB
Dimensions:                     (nodes: 2, techs: 7, carriers: 3,
                                 timesteps: 744, costs: 1)
Coordinates:
  * techs                       (techs) object 56B 'aluminium_demand' ... 're...
  * nodes                       (nodes) <U11 88B 'Iceland' 'Netherlands'
  * carriers                    (carriers) <U11 132B 'aluminium' ... 'fuel'
  * timesteps                   (timesteps) datetime64[ns] 6kB 2005-01-01 ......
  * costs                       (costs) object 8B 'monetary'
Data variables: (12/16)
    flow_cap                    (nodes, techs, carriers) float64 336B nan ......
    link_flow_cap               (techs) float64 56B nan nan 3e+03 ... nan nan
    flow_out                    (nodes, techs, carriers, timesteps) float64 250kB ...
    flow_in                     (nodes, techs, carriers, timesteps) float64 250kB ...
    source_use                  (nodes, techs, timesteps) float64 83kB nan .....
    source_cap                  (nodes, techs) float64 112B nan nan ... nan nan
    ...                          ...
    cost_operation_variable     (nodes, techs, costs, timesteps) float64 83kB ...
    cost                        (nodes, techs, costs) float64 112B nan ... nan
    capacity_factor             (nodes, techs, carriers, timesteps) float64 250kB ...
    systemwide_capacity_factor  (techs, carriers) float64 168B 0.0 0.0 ... 0.0
    systemwide_levelised_cost   (carriers, techs, costs) float64 168B nan ......
    total_levelised_cost        (carriers, costs) float64 24B -1.344 ... -3.872
Attributes: (12/13)
    termination_condition:         optimal
    calliope_version_defined:      0.7.0
    calliope_version_initialised:  0.7.0.dev6
    applied_overrides:             
    scenario:                      None
    defaults:                      {'bigM': 1000000000.0, 'objective_cost_wei...
    ...                            ...
    name:                          main model
    timestamp_model_creation:      1753879056.755022
    timestamp_build_start:         1753879057.518692
    timestamp_build_complete:      1753879061.752128
    timestamp_solve_start:         1753879061.752128
    timestamp_solve_complete:      1753879062.644189

In [7]:
costs = model.results.cost.to_series().dropna()
costs.head()

nodes        techs                  costs   
Iceland      fuel_supply            monetary        2.6040
             renewable_electricity  monetary       39.0625
Netherlands  aluminium_demand       monetary   -15000.0000
             fuel_necessity         monetary    -5208.0000
Name: cost, dtype: float64

In [8]:
lcoes = model.results.systemwide_levelised_cost.to_series().dropna()
lcoes

carriers     techs                  costs   
electricity  renewable_electricity  monetary    0.0025
fuel         fuel_supply            monetary    0.0005
Name: systemwide_levelised_cost, dtype: float64

In [9]:
flow_out = model.results.flow_out.sel(carriers="aluminium").where(lambda x: x != 0).to_series().dropna()
flow_out

nodes        techs                     timesteps          
Iceland      aluminium_smelter         2005-01-01 00:00:00    3000.0
                                       2005-01-07 02:00:00    3000.0
                                       2005-01-13 04:00:00    3000.0
                                       2005-01-19 05:00:00    3000.0
                                       2005-01-25 08:00:00    3000.0
Netherlands  aluminium_transport_tech  2005-01-04 00:00:00    3000.0
                                       2005-01-10 02:00:00    3000.0
                                       2005-01-16 04:00:00    3000.0
                                       2005-01-22 05:00:00    3000.0
                                       2005-01-28 08:00:00    3000.0
Name: flow_out, dtype: float64

In [10]:
colors = model.inputs.color.to_series().to_dict()

In [11]:
model.to_netcdf("main_model.nc")